In [ ]:
#This version works

import cv2
import depthai as dai
import numpy as np
import blobconverter

# Create DepthAI pipeline
pipeline = dai.Pipeline()

# Define camera nodes
cam_rgb   = pipeline.create(dai.node.ColorCamera)
mono_left  = pipeline.create(dai.node.MonoCamera)
mono_right = pipeline.create(dai.node.MonoCamera)
stereo     = pipeline.create(dai.node.StereoDepth)
detection_nn = pipeline.create(dai.node.YoloSpatialDetectionNetwork)  # Spatial NN for YOLO

# Define XLink outputs for streaming frames and detections to host
xout_rgb   = pipeline.create(dai.node.XLinkOut)
xout_depth = pipeline.create(dai.node.XLinkOut)
xout_nn    = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
xout_depth.setStreamName("depth")
xout_nn.setStreamName("detections")

# Camera configuration (RGB camera)
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setPreviewSize(416, 416)  # neural network input size for TinyYOLOv4
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

# Mono cameras (for depth)
mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
mono_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)

# Stereo depth configuration
stereo.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
stereo.setDepthAlign(dai.CameraBoardSocket.RGB)        # Align depth map to RGB camera perspective:contentReference[oaicite:25]{index=25}
stereo.setOutputSize(mono_left.getResolutionWidth(), 
                     mono_left.getResolutionHeight())
stereo.setSubpixel(True)  # improve depth precision

# Download and set neural network model (Tiny-YOLOv4 COCO 416x416)
blob_path = blobconverter.from_zoo(name="yolov4_tiny_coco_416x416", 
                                   zoo_type="depthai", 
                                   shaves=6)
detection_nn.setBlobPath(str(blob_path))
detection_nn.setConfidenceThreshold(0.5)
detection_nn.input.setBlocking(False)
detection_nn.setBoundingBoxScaleFactor(0.5)  # extra area for depth averaging
detection_nn.setDepthLowerThreshold(100)     # 100 mm -> 10 cm minimum distance:contentReference[oaicite:26]{index=26}
detection_nn.setDepthUpperThreshold(5000)    # 5000 mm -> 5 m maximum distance:contentReference[oaicite:27]{index=27}

# YOLO-specific network settings (for COCO Tiny-YOLOv4 416x416)
detection_nn.setNumClasses(80)
detection_nn.setCoordinateSize(4)
detection_nn.setAnchors([10,14, 23,27, 37,58, 81,82, 135,169, 344,319])       # from model config:contentReference[oaicite:28]{index=28}
detection_nn.setAnchorMasks({ "side26": [1,2,3], "side13": [3,4,5] })         # YOLOv4-tiny masks:contentReference[oaicite:29]{index=29}
detection_nn.setIouThreshold(0.5)

# Link nodes: RGB -> Neural Network, Mono -> StereoDepth, Depth -> Neural Network
cam_rgb.preview.link(detection_nn.input)
mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)
stereo.depth.link(detection_nn.inputDepth)

# Link NN outputs to XLink outputs
detection_nn.passthrough.link(xout_rgb.input)      # passthrough RGB frames (frames that went into NN)
detection_nn.passthroughDepth.link(xout_depth.input)  # aligned depth frames
detection_nn.out.link(xout_nn.input)              # detection outputs (bounding boxes + coordinates)

# Initialize the device and pipeline
with dai.Device(pipeline) as device:
    # Output queues to retrieve frames and detections
    q_rgb   = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    q_depth = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    q_det   = device.getOutputQueue(name="detections", maxSize=4, blocking=False)

    # Get label names for COCO classes (for display purposes)
    label_map = [
        "person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat",
        "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
        "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella",
        "handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
        "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle",
        "wine glass","cup","fork","knife","spoon","bowl","banana","apple","sandwich",
        "orange","broccoli","carrot","hot dog","pizza","donut","cake","chair","sofa",
        "pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse","remote",
        "keyboard","cell phone","microwave","oven","toaster","sink","refrigerator","book",
        "clock","vase","scissors","teddy bear","hair drier","toothbrush"
    ]  # COCO 80-class label list (indices 0-79):contentReference[oaicite:30]{index=30}:contentReference[oaicite:31]{index=31}

    while True:
        in_rgb   = q_rgb.get()      # latest RGB frame
        in_depth = q_depth.get()    # latest depth frame (aligned to RGB)
        in_dets  = q_det.get()      # latest detection results

        frame = in_rgb.getCvFrame()               # OpenCV BGR frame from color camera
        depth_frame = in_depth.getFrame()         # depth data in millimeters
        detections = in_dets.detections           # list of spatial detections

        # (Optional) Colorize depth map for visualization
        # Normalize depth for display, then apply colormap
        #depth_min = depth_frame.min()
        #depth_max = depth_frame.max()
        #depth_frame_disp = ((depth_frame - depth_min) / (depth_max - depth_min) * 255).astype(np.uint8)
        #depth_frame_disp = cv2.applyColorMap(depth_frame_disp, cv2.COLORMAP_JET)

        # Iterate over detections and draw bounding boxes and labels
        for det in detections:
            
            # Get bounding box coordinates (normalized 0..1 from NN, convert to pixel coords)
            x1 = int(det.xmin * frame.shape[1])
            y1 = int(det.ymin * frame.shape[0])
            x2 = int(det.xmax * frame.shape[1])
            y2 = int(det.ymax * frame.shape[0])

            # Draw rectangle on RGB frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

            # Determine label text to display
            label = str(det.label)
            if det.label < len(label_map):
                label = label_map[det.label]
            conf  = int(det.confidence * 100)  # confidence percentage

            # Draw label and confidence
            cv2.putText(frame, f"{label} ({conf}%)", (x1+5, y1+20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            # Draw spatial coordinates (X, Y, Z in mm)
            coords = det.spatialCoordinates  # spatial coordinates relative to camera
            cv2.putText(frame, f"X: {int(coords.x)} mm", (x1+5, y1+35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            cv2.putText(frame, f"Y: {int(coords.y)} mm", (x1+5, y1+50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            cv2.putText(frame, f"Z: {int(coords.z)} mm", (x1+5, y1+65),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            if label == "cup":
                print(label, ":",coords.x,coords.y,coords.z)
            # Note: X is positive to the right, Y is positive downwards, Z is depth (forward):contentReference[oaicite:32]{index=32}

        # Show the frames in windows
        cv2.imshow("RGB", frame)
        #cv2.imshow("Depth", depth_frame_disp)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
cv2.destroyAllWindows()
#test

C:\Users\david\AppData\Local\Temp\ipykernel_15344\2963372548.py:27: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
C:\Users\david\AppData\Local\Temp\ipykernel_15344\2963372548.py:34: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
C:\Users\david\AppData\Local\Temp\ipykernel_15344\2963372548.py:35: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
C:\Users\david\AppData\Local\Temp\ipykernel_15344\2963372548.py:41: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  stereo.setDepthAlign(dai.CameraBoardSocket.RGB)        # Align depth map to RGB camera perspective:contentReference[oaicite:25]{index=25}


cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -0.0 -0.0 0.0
cup : -78.48713684082031 -252.96307373046875 995.7173461914062
cup : -80.81514739990234 -262.2625427246094 1025.2513427734375
cup : -80.81514739990234 -260.4662170410156 1025.2513427734375
cup : -81.15904235839844 -259.7706298828125 1029.6141357421875
cup : -79.35549926757812 -261.5745849609375 1029.6141357421875
cup : -82.61103057861328 -260.4662170410156 1025.2513427734375
cup : -82.61103057861328 -258.6698913574219 1025.2513427734375
cup : -82.61103057861328 -260.4662170410156 1025.2513427734375
cup : -82.61103057861328 -260.4662170410156 1025.2513427734375
cup : -82.61103057861328 -260.4662170410156 1025.2513427734375
cup : -80.81514739990234 -262.2625427246094 1025.2513427734375
cup : -80.81514739990234 -260.4662170410156 1025.2513427734375
cup : -80.81514739990234 -260.4662170410156 1025.2513427734375
cup : -82.26245880126

In [2]:
import cv2
import depthai as dai
import numpy as np
import blobconverter

# Create DepthAI pipeline
pipeline = dai.Pipeline()

# Define camera nodes
cam_rgb   = pipeline.create(dai.node.ColorCamera)
mono_left  = pipeline.create(dai.node.MonoCamera)
mono_right = pipeline.create(dai.node.MonoCamera)
stereo     = pipeline.create(dai.node.StereoDepth)
detection_nn = pipeline.create(dai.node.YoloSpatialDetectionNetwork)  # Spatial NN for YOLO

# Define XLink outputs for streaming frames and detections to host
xout_rgb   = pipeline.create(dai.node.XLinkOut)
xout_depth = pipeline.create(dai.node.XLinkOut)
xout_nn    = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
xout_depth.setStreamName("depth")
xout_nn.setStreamName("detections")

# Camera configuration (RGB camera)
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setPreviewSize(416, 416)  # neural network input size for TinyYOLOv4
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

# Mono cameras (for depth)
mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
mono_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)

# Stereo depth configuration
stereo.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
stereo.setDepthAlign(dai.CameraBoardSocket.RGB)        # Align depth map to RGB camera perspective:contentReference[oaicite:25]{index=25}
stereo.setOutputSize(mono_left.getResolutionWidth(), 
                     mono_left.getResolutionHeight())
stereo.setSubpixel(True)  # improve depth precision

# Download and set neural network model (Tiny-YOLOv4 COCO 416x416)
blob_path = blobconverter.from_zoo(name="yolov4_tiny_coco_416x416", 
                                   zoo_type="depthai", 
                                   shaves=6)
detection_nn.setBlobPath(str(blob_path))
detection_nn.setConfidenceThreshold(0.5)
detection_nn.input.setBlocking(False)
detection_nn.setBoundingBoxScaleFactor(0.5)  # extra area for depth averaging
detection_nn.setDepthLowerThreshold(100)     # 100 mm -> 10 cm minimum distance:contentReference[oaicite:26]{index=26}
detection_nn.setDepthUpperThreshold(5000)    # 5000 mm -> 5 m maximum distance:contentReference[oaicite:27]{index=27}

# YOLO-specific network settings (for COCO Tiny-YOLOv4 416x416)
detection_nn.setNumClasses(80)
detection_nn.setCoordinateSize(4)
detection_nn.setAnchors([10,14, 23,27, 37,58, 81,82, 135,169, 344,319])       # from model config:contentReference[oaicite:28]{index=28}
detection_nn.setAnchorMasks({ "side26": [1,2,3], "side13": [3,4,5] })         # YOLOv4-tiny masks:contentReference[oaicite:29]{index=29}
detection_nn.setIouThreshold(0.5)

# Link nodes: RGB -> Neural Network, Mono -> StereoDepth, Depth -> Neural Network
cam_rgb.preview.link(detection_nn.input)
mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)
stereo.depth.link(detection_nn.inputDepth)

# Link NN outputs to XLink outputs
detection_nn.passthrough.link(xout_rgb.input)      # passthrough RGB frames (frames that went into NN)
detection_nn.passthroughDepth.link(xout_depth.input)  # aligned depth frames
detection_nn.out.link(xout_nn.input)              # detection outputs (bounding boxes + coordinates)

# Initialize the device and pipeline
with dai.Device(pipeline) as device:
    # Output queues to retrieve frames and detections
    q_rgb   = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    q_depth = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    q_det   = device.getOutputQueue(name="detections", maxSize=4, blocking=False)

    # Get label names for COCO classes (for display purposes)
    label_map = [
        "person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat",
        "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
        "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella",
        "handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
        "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle",
        "wine glass","cup","fork","knife","spoon","bowl","banana","apple","sandwich",
        "orange","broccoli","carrot","hot dog","pizza","donut","cake","chair","sofa",
        "pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse","remote",
        "keyboard","cell phone","microwave","oven","toaster","sink","refrigerator","book",
        "clock","vase","scissors","teddy bear","hair drier","toothbrush"
    ]  # COCO 80-class label list (indices 0-79):contentReference[oaicite:30]{index=30}:contentReference[oaicite:31]{index=31}

    while True:
        in_rgb   = q_rgb.get()      # latest RGB frame
        in_depth = q_depth.get()    # latest depth frame (aligned to RGB)
        in_dets  = q_det.get()      # latest detection results

        frame = in_rgb.getCvFrame()               # OpenCV BGR frame from color camera
        depth_frame = in_depth.getFrame()         # depth data in millimeters
        detections = in_dets.detections           # list of spatial detections

        # (Optional) Colorize depth map for visualization
        # Normalize depth for display, then apply colormap
        depth_min = depth_frame.min()
        depth_max = depth_frame.max()
        depth_frame_disp = ((depth_frame - depth_min) / (depth_max - depth_min) * 255).astype(np.uint8)
        depth_frame_disp = cv2.applyColorMap(depth_frame_disp, cv2.COLORMAP_JET)

        # Iterate over detections and draw bounding boxes and labels
        for det in detections:
            
                        # Determine label text to display
            label = str(det.label)
            if det.label < len(label_map):
                label = label_map[det.label]
            conf  = int(det.confidence * 100)  # confidence percentage

            if label == "cup":
                # Get bounding box coordinates (normalized 0..1 from NN, convert to pixel coords)
                x1 = int(det.xmin * frame.shape[1])
                y1 = int(det.ymin * frame.shape[0])
                x2 = int(det.xmax * frame.shape[1])
                y2 = int(det.ymax * frame.shape[0])

                # Draw rectangle on RGB frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

                # Draw label and confidence
                cv2.putText(frame, f"{label} ({conf}%)", (x1+5, y1+20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                # Draw spatial coordinates (X, Y, Z in mm)
                coords = det.spatialCoordinates  # spatial coordinates relative to camera
                cv2.putText(frame, f"X: {int(coords.x)} mm", (x1+5, y1+35),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
                cv2.putText(frame, f"Y: {int(coords.y)} mm", (x1+5, y1+50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
                cv2.putText(frame, f"Z: {int(coords.z)} mm", (x1+5, y1+65),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                print(label, ":",coords.x,coords.y,coords.z)
            # Note: X is positive to the right, Y is positive downwards, Z is depth (forward):contentReference[oaicite:32]{index=32}

        # Show the frames in windows
        cv2.imshow("RGB", frame)
        cv2.imshow("Depth", depth_frame_disp)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
cv2.destroyAllWindows()
#test

C:\Users\david\AppData\Local\Temp\ipykernel_15344\2432229076.py:25: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
C:\Users\david\AppData\Local\Temp\ipykernel_15344\2432229076.py:32: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
C:\Users\david\AppData\Local\Temp\ipykernel_15344\2432229076.py:33: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
C:\Users\david\AppData\Local\Temp\ipykernel_15344\2432229076.py:39: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  stereo.setDepthAlign(dai.CameraBoardSocket.RGB)        # Align depth map to RGB camera perspective:contentReference[oaicite:25]{index=25}


cup : -137.61941528320312 -156.07630920410156 2638.941162109375
cup : -22.457515716552734 -153.34527587890625 759.9483032226562
cup : -28.08502197265625 -141.7995147705078 734.38427734375
cup : -30.176788330078125 -139.6639862060547 723.3242797851562
cup : -32.86014938354492 -116.91671752929688 700.1272583007812
cup : -34.68745422363281 -107.72501373291016 688.0906982421875
cup : -35.74810028076172 -84.2391128540039 663.3802490234375
cup : -37.53597640991211 -75.12377166748047 654.3424072265625
cup : -36.443660736083984 -58.57112121582031 635.3006591796875
cup : -36.98884582519531 -29.611501693725586 607.9586181640625
cup : -46.17344284057617 9.429654121398926 542.0850219726562
cup : -44.82637023925781 7.476218223571777 537.234130859375
cup : -46.27999496459961 7.409910202026367 532.4692993164062
cup : -46.48411178588867 4.56041145324707 524.3311157226562
cup : -46.0816535949707 3.61674165725708 519.7914428710938
cup : -45.20111083984375 1.7738159894943237 509.859130859375
cup : -46.67

KeyboardInterrupt: 

In [ ]:
import time, json

# Find the index of 'cup' in our label map (fallback to name match at runtime)
try:
    CUP_LABEL_INDEX = [i for i, n in enumerate(label_map) if n.lower() == "cup"][0]
except:
    CUP_LABEL_INDEX = None

frame_id = 0
log_path = "cup_detections.jsonl"
log_file = open(log_path, "a", buffering=1)  # line-buffered

while True:
    in_rgb   = q_rgb.get()
    in_depth = q_depth.get()
    in_dets  = q_det.get()

    frame = in_rgb.getCvFrame()
    depth_frame = in_depth.getFrame()
    detections = in_dets.detections

    # ---- draw & visualize (same as before) ----
    for det in detections:
        x1 = int(det.xmin * frame.shape[1])
        y1 = int(det.ymin * frame.shape[0])
        x2 = int(det.xmax * frame.shape[1])
        y2 = int(det.ymax * frame.shape[0])

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

        name = label_map[det.label] if det.label < len(label_map) else str(det.label)
        conf = int(det.confidence * 100)
        cv2.putText(frame, f"{name} ({conf}%)", (x1+5, y1+20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        coords = det.spatialCoordinates
        cv2.putText(frame, f"X:{int(coords.x)} Y:{int(coords.y)} Z:{int(coords.z)} mm",
                    (x1+5, y1+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    # ---- build a JSON record with ONLY 'cup' detections ----
    cups = []
    for det in detections:
        name = label_map[det.label] if det.label < len(label_map) else str(det.label)
        if (CUP_LABEL_INDEX is not None and det.label == CUP_LABEL_INDEX) or (name.lower() == "cup"):
            cups.append({
                "label": "cup",
                "conf": round(float(det.confidence), 3),
                "bbox": {
                    "x1": int(det.xmin * frame.shape[1]),
                    "y1": int(det.ymin * frame.shape[0]),
                    "x2": int(det.xmax * frame.shape[1]),
                    "y2": int(det.ymax * frame.shape[0]),
                },
                "spatial_mm": {
                    "x": int(det.spatialCoordinates.x),
                    "y": int(det.spatialCoordinates.y),
                    "z": int(det.spatialCoordinates.z)
                }
            })

    record = {
        "ts": time.time(),
        "frame_id": frame_id,
        "cups": cups
    }
    log_file.write(json.dumps(record) + "\n")

    # ---- show windows (optional) ----
    cv2.imshow("RGB", frame)
    # (Optional) visualize depth as before if you kept that code
    # cv2.imshow("Depth", depth_frame_disp)

    frame_id += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

log_file.close()
cv2.destroyAllWindows()


C:\Users\david\AppData\Local\Temp\ipykernel_51580\1203156475.py:25: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
C:\Users\david\AppData\Local\Temp\ipykernel_51580\1203156475.py:32: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
C:\Users\david\AppData\Local\Temp\ipykernel_51580\1203156475.py:33: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
C:\Users\david\AppData\Local\Temp\ipykernel_51580\1203156475.py:39: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  stereo.setDepthAlign(dai.CameraBoardSocket.RGB)        # Align depth map to RGB camera perspective:contentReference[oaicite:25]{index=25}
C:\Users\david\AppData\Local\Temp\ipykernel_51580\1203156475.py:107: RuntimeWarning